In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 基本ツール
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display
import matplotlib.ticker as ticker
import seaborn as sns
import time

import lightgbm as lgb

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import  r2_score

# optuna

!pip install optuna
import optuna.integration.lightgbm as lgb

     |████████████████████████████████| 296kB 17.4MB/s 
     |████████████████████████████████| 81kB 12.9MB/s 
     |████████████████████████████████| 1.2MB 59.0MB/s 
     |████████████████████████████████| 143kB 57.2MB/s 
     |████████████████████████████████| 112kB 60.8MB/s 
     |████████████████████████████████| 51kB 9.1MB/s 
     |████████████████████████████████| 481kB 40.6MB/s 
  Created wheel for alembic: filename=alembic-1.5.5-py2.py3-none-any.whl size=156597 sha256=0140721b18effc395a0b8800ee4339fe4fcb8ada87a5ede8b3fe149664ecf4c0
  Stored in directory: /root/.cache/pip/wheels/74/3f/61/7de6e3cef766d2680a5d81b1a388286e640f6a681eb589d643
  Created wheel for Mako: filename=Mako-1.1.4-py2.py3-none-any.whl size=75675 sha256=15fffe37b6bf0eb42827d3a792c24758e9d7d3fffed492a66104720e9372862b
  Stored in directory: /root/.cache/pip/wheels/ad/10/d3/aeb26e20d19045e2a68e5d3cbb57432e11b5d9c92c99f98d47
  Created wheel for pyperclip: filename=pyperclip-1.8.2-cp36-none-any.whl size=11108 sha25

In [3]:
# GPU確認

!nvidia-smi


Tue Feb 23 05:58:46 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
# 時間計測

def tic():
    # require to import time
    global start_time_tictoc
    start_time_tictoc = time.time()
    
    
def toc(tag = 'elapsed time'):
    if 'start_time_tictoc' in globals():
        print('{}: {:.5f} sec'.format(tag, time.time() - start_time_tictoc), end = '' )
    else:
        print('tic has not been called')
        
        


In [5]:
raw_data   = pd.read_csv('/content/drive/MyDrive/kaggle/train.csv')
raw_data_  = pd.read_csv('/content/drive/MyDrive/kaggle/test.csv')
sample_submission  = pd.read_csv('/content/drive/MyDrive/kaggle/sample_submission.csv')
train_data = raw_data.copy()
test_data  = raw_data_.copy()

In [6]:
train_data.head(100)

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
0,1,A,B,A,A,B,D,A,E,C,I,0.923191,0.684968,0.124454,0.217886,0.281421,0.881122,0.421650,0.741413,0.895799,0.802461,0.724417,0.701915,0.877618,0.719903,6.994023
1,2,B,A,A,A,B,B,A,E,A,F,0.437627,0.014213,0.357438,0.846127,0.282354,0.440011,0.346230,0.278495,0.593413,0.546056,0.613252,0.741289,0.326679,0.808464,8.071256
2,3,A,A,A,C,B,D,A,B,C,N,0.732209,0.760122,0.454644,0.812990,0.293756,0.914155,0.369602,0.832564,0.865620,0.825251,0.264104,0.695561,0.869133,0.828352,5.760456
3,4,A,A,A,C,B,D,A,E,G,K,0.705142,0.771678,0.153735,0.732893,0.769785,0.934138,0.578930,0.407313,0.868099,0.794402,0.494269,0.698125,0.809799,0.614766,7.806457
4,6,A,B,A,A,B,B,A,E,C,F,0.486063,0.639349,0.496212,0.354186,0.279105,0.382600,0.705940,0.325193,0.440967,0.462146,0.724447,0.683073,0.343457,0.297743,6.868974
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,164,A,B,A,C,B,B,A,E,E,F,0.283145,0.156583,0.531322,0.180019,0.647067,0.198953,0.304955,0.228038,0.165184,0.350502,0.231549,0.265881,0.351718,0.251234,8.574307
96,165,A,B,A,D,B,B,A,E,C,H,0.533576,0.670130,0.336240,0.159201,0.278283,0.390233,0.406308,0.254529,0.401283,0.443222,0.611084,0.373161,0.329228,0.289662,7.613036
97,167,A,B,A,C,B,D,A,E,E,B,0.347727,0.674733,0.493068,0.863647,0.754639,0.435150,0.340275,0.260918,0.311286,0.611101,0.568261,0.397329,0.650514,0.805869,7.691082
98,168,A,B,A,C,C,D,A,E,G,F,0.871552,0.745887,0.311748,0.648881,0.277171,0.679719,0.480595,0.484355,0.913394,0.840664,0.577120,0.346455,0.893273,0.671947,8.441794


In [7]:
def kesson_table(df):
    null_val = df.isnull().sum()
    percent  = 100 * df.isnull().sum() / len(df)
    kesson_table = pd.concat([null_val, percent], axis=1)
    kesson_table_ren_columns = kesson_table.rename(columns = {0 : '欠損数', 1 : '%'})
    return kesson_table_ren_columns

print('train_data')
display(kesson_table(train_data))
display(kesson_table(test_data))

train_data


,欠損数,%
id,0,0.0
cat0,0,0.0
cat1,0,0.0
cat2,0,0.0
cat3,0,0.0
cat4,0,0.0
cat5,0,0.0
cat6,0,0.0
cat7,0,0.0
cat8,0,0.0


,欠損数,%
id,0,0.0
cat0,0,0.0
cat1,0,0.0
cat2,0,0.0
cat3,0,0.0
cat4,0,0.0
cat5,0,0.0
cat6,0,0.0
cat7,0,0.0
cat8,0,0.0


In [8]:
X = train_data.iloc[:, 1:25]
y = train_data.iloc[:,25:]

X_test = test_data.iloc[:,1:25]

In [9]:
print(X)
print(X_test)

       cat0 cat1 cat2 cat3  ...    cont10    cont11    cont12    cont13
0         A    B    A    A  ...  0.724417  0.701915  0.877618  0.719903
1         B    A    A    A  ...  0.613252  0.741289  0.326679  0.808464
2         A    A    A    C  ...  0.264104  0.695561  0.869133  0.828352
3         A    A    A    C  ...  0.494269  0.698125  0.809799  0.614766
4         A    B    A    A  ...  0.724447  0.683073  0.343457  0.297743
...     ...  ...  ...  ...  ...       ...       ...       ...       ...
299995    A    B    A    C  ...  0.235711  0.383477  0.215227  0.793630
299996    A    B    A    C  ...  0.286094  0.324874  0.306933  0.230902
299997    A    B    A    C  ...  0.235788  0.316671  0.250286  0.349041
299998    A    B    B    C  ...  0.389391  0.245234  0.303895  0.481138
299999    A    A    B    A  ...  0.751251  0.608412  0.712868  0.452400

[300000 rows x 24 columns]
       cat0 cat1 cat2 cat3  ...    cont10    cont11    cont12    cont13
0         A    B    A    C  ...  0.2

In [10]:
print(y)

          target
0       6.994023
1       8.071256
2       5.760456
3       7.806457
4       6.868974
...          ...
299995  8.343538
299996  7.851861
299997  7.600558
299998  8.272095
299999  6.025685

[300000 rows x 1 columns]


In [11]:
#　ダミー化
X = pd.get_dummies(X, columns=['cat0','cat1','cat2','cat3','cat4','cat5','cat6','cat7','cat8','cat9'])
X_test = pd.get_dummies(X_test, columns=['cat0','cat1','cat2','cat3','cat4','cat5','cat6','cat7','cat8','cat9'])

X = X.drop('cat6_G',axis=1)


In [12]:
display(X)
display(X_test)

,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cat0_A,cat0_B,cat1_A,cat1_B,cat2_A,cat2_B,cat3_A,cat3_B,cat3_C,cat3_D,cat4_A,cat4_B,cat4_C,cat4_D,cat5_A,cat5_B,cat5_C,cat5_D,cat6_A,cat6_B,cat6_C,cat6_D,cat6_E,cat6_H,cat6_I,cat7_A,cat7_B,cat7_C,cat7_D,cat7_E,cat7_F,cat7_G,cat7_I,cat8_A,cat8_B,cat8_C,cat8_D,cat8_E,cat8_F,cat8_G,cat9_A,cat9_B,cat9_C,cat9_D,cat9_E,cat9_F,cat9_G,cat9_H,cat9_I,cat9_J,cat9_K,cat9_L,cat9_M,cat9_N,cat9_O
0,0.923191,0.684968,0.124454,0.217886,0.281421,0.881122,0.421650,0.741413,0.895799,0.802461,0.724417,0.701915,0.877618,0.719903,1,0,0,1,1,0,1,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,0.437627,0.014213,0.357438,0.846127,0.282354,0.440011,0.346230,0.278495,0.593413,0.546056,0.613252,0.741289,0.326679,0.808464,0,1,1,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,0.732209,0.760122,0.454644,0.812990,0.293756,0.914155,0.369602,0.832564,0.865620,0.825251,0.264104,0.695561,0.869133,0.828352,1,0,1,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,0.705142,0.771678,0.153735,0.732893,0.769785,0.934138,0.578930,0.407313,0.868099,0.794402,0.494269,0.698125,0.809799,0.614766,1,0,1,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,0.486063,0.639349,0.496212,0.354186,0.279105,0.382600,0.705940,0.325193,0.440967,0.462146,0.724447,0.683073,0.343457,0.297743,1,0,0,1,1,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,0.260716,0.712438,0.161661,0.442794,0.768447,0.269578,0.258655,0.363598,0.300619,0.340516,0.235711,0.383477,0.215227,0.793630,1,0,0,1,1,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
299996,0.173302,0.121591,0.592514,0.193711,0.775951,0.197211,0.257024,0.574304,0.227035,0.322583,0.286094,0.324874,0.306933,0.230902,1,0,0,1,1,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
299997,0.342856,0.617869,0.462991,0.418098,0.297406,0.449482,0.386172,0.476217,0.135947,0.502730,0.235788,0.316671,0.250286,0.349041,1,0,0,1,1,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
299998,0.599403,0.686054,0.660860,0.187199,0.758642,0.363130,0.324132,0.229017,0.220888,0.515304,0.389391,0.245234,0.303895,0.481138,1,0,0,1,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cat0_A,cat0_B,cat1_A,cat1_B,cat2_A,cat2_B,cat3_A,cat3_B,cat3_C,cat3_D,cat4_A,cat4_B,cat4_C,cat4_D,cat5_A,cat5_B,cat5_C,cat5_D,cat6_A,cat6_B,cat6_C,cat6_D,cat6_E,cat6_H,cat6_I,cat7_A,cat7_B,cat7_C,cat7_D,cat7_E,cat7_F,cat7_G,cat7_I,cat8_A,cat8_B,cat8_C,cat8_D,cat8_E,cat8_F,cat8_G,cat9_A,cat9_B,cat9_C,cat9_D,cat9_E,cat9_F,cat9_G,cat9_H,cat9_I,cat9_J,cat9_K,cat9_L,cat9_M,cat9_N,cat9_O
0,0.331261,0.551131,0.486284,0.156058,0.701679,0.595507,0.286912,0.279884,0.202234,0.242654,0.285147,0.264308,0.653654,0.302448,1,0,0,1,1,0,0,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,0.402404,0.617546,0.296852,0.187440,0.277480,0.479552,0.397436,0.476742,0.857073,0.516393,0.562065,0.730542,0.318492,0.736251,1,0,0,1,1,0,0,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,0.530677,0.618126,0.760865,0.199872,0.279508,0.676395,0.695284,0.253316,0.586934,0.548555,0.836193,0.759788,0.333572,0.273905,1,0,0,1,1,0,0,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,0.439653,0.487403,0.715786,0.192808,0.479503,0.759875,0.240049,0.298074,0.442475,0.596746,0.414131,0.255382,0.589080,0.311625,1,0,1,0,0,1,1,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,0.389844,0.752760,0.315269,0.191734,0.757845,0.210232,0.329851,0.616663,0.170475,0.263235,0.710961,0.224045,0.285860,0.794931,1,0,0,1,1,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,0.919265,0.111147,0.199583,0.181354,0.277365,0.963678,0.240482,0.686462,0.915165,0.848878,0.459598,0.590327,0.864873,0.425258,1,0,1,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
199996,0.362875,-0.020197,0.469025,0.336185,0.523174,0.232072,0.363421,0.694092,0.137002,0.319465,0.364527,0.388908,0.664357,0.224215,1,0,1,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
199997,0.463060,0.740421,0.446293,0.411387,0.517103,0.432927,0.811876,0.328398,0.496017,0.538779,0.466338,0.643869,0.749590,0.457702,1,0,1,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
199998,0.708709,0.418490,0.193004,0.862700,0.279153,0.837712,0.680886,0.534439,0.501588,0.809053,0.631704,0.766426,0.937139,0.796304,1,0,0,1,1,0,1,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [13]:
# 正規化
X = (X - np.min(X)) / (np.max(X) - np.min(X))
X_test = (X_test - np.min(X_test)) / (np.max(X_test) - np.min(X_test)) 

In [14]:
display(X)
display(X_test)

,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cat0_A,cat0_B,cat1_A,cat1_B,cat2_A,cat2_B,cat3_A,cat3_B,cat3_C,cat3_D,cat4_A,cat4_B,cat4_C,cat4_D,cat5_A,cat5_B,cat5_C,cat5_D,cat6_A,cat6_B,cat6_C,cat6_D,cat6_E,cat6_H,cat6_I,cat7_A,cat7_B,cat7_C,cat7_D,cat7_E,cat7_F,cat7_G,cat7_I,cat8_A,cat8_B,cat8_C,cat8_D,cat8_E,cat8_F,cat8_G,cat9_A,cat9_B,cat9_C,cat9_D,cat9_E,cat9_F,cat9_G,cat9_H,cat9_I,cat9_J,cat9_K,cat9_L,cat9_M,cat9_N,cat9_O
0,0.887037,0.816091,0.171367,0.095360,0.114564,0.855691,0.359931,0.643496,0.882829,0.811322,0.685241,0.654854,0.913387,0.785206,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.463397,0.076438,0.387501,0.816060,0.115723,0.465907,0.288058,0.084307,0.583471,0.526128,0.570654,0.695283,0.343629,0.908989,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.720411,0.898964,0.477677,0.778047,0.129890,0.884880,0.310331,0.753604,0.852951,0.836672,0.210755,0.648330,0.904611,0.936786,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.696796,0.911708,0.198531,0.686162,0.721352,0.902538,0.509813,0.239914,0.855406,0.802359,0.448007,0.650962,0.843251,0.638256,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.505656,0.765785,0.516239,0.251719,0.111686,0.415176,0.630849,0.140716,0.432551,0.432796,0.685273,0.635506,0.360980,0.195151,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,0.309047,0.846382,0.205883,0.353368,0.719690,0.315305,0.204602,0.187108,0.293608,0.297509,0.181488,0.327877,0.228370,0.888255,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
299996,0.232781,0.194846,0.605575,0.067627,0.729014,0.251359,0.203048,0.441633,0.220760,0.277562,0.233423,0.267703,0.323208,0.101727,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
299997,0.380712,0.742099,0.485420,0.325038,0.134425,0.474276,0.326122,0.323148,0.130585,0.477937,0.181568,0.259280,0.264627,0.266850,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
299998,0.604542,0.817289,0.668978,0.060156,0.707507,0.397972,0.267000,0.024539,0.214676,0.491923,0.339900,0.185928,0.320067,0.451483,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cat0_A,cat0_B,cat1_A,cat1_B,cat2_A,cat2_B,cat3_A,cat3_B,cat3_C,cat3_D,cat4_A,cat4_B,cat4_C,cat4_D,cat5_A,cat5_B,cat5_C,cat5_D,cat6_A,cat6_B,cat6_C,cat6_D,cat6_E,cat6_H,cat6_I,cat7_A,cat7_B,cat7_C,cat7_D,cat7_E,cat7_F,cat7_G,cat7_I,cat8_A,cat8_B,cat8_C,cat8_D,cat8_E,cat8_F,cat8_G,cat9_A,cat9_B,cat9_C,cat9_D,cat9_E,cat9_F,cat9_G,cat9_H,cat9_I,cat9_J,cat9_K,cat9_L,cat9_M,cat9_N,cat9_O
0,0.371186,0.670395,0.503261,0.023265,0.646458,0.594293,0.237411,0.082901,0.198780,0.181973,0.230258,0.200179,0.672012,0.202090,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.433672,0.744706,0.327461,0.059415,0.110513,0.488834,0.342425,0.318204,0.844855,0.493316,0.515667,0.681469,0.320500,0.801660,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.546334,0.745355,0.758084,0.073736,0.113075,0.667858,0.625424,0.051145,0.578331,0.529895,0.798202,0.711660,0.336316,0.162641,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.466387,0.599089,0.716249,0.065598,0.365754,0.743781,0.192884,0.104644,0.435806,0.584707,0.363196,0.190965,0.604288,0.214774,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.422640,0.895997,0.344553,0.064361,0.717419,0.243894,0.278209,0.485452,0.167447,0.205381,0.669130,0.158616,0.286277,0.882763,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,0.887630,0.178095,0.237192,0.052404,0.110368,0.929135,0.193296,0.568882,0.902169,0.871474,0.410059,0.536726,0.893534,0.371829,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
199996,0.398954,0.031134,0.487245,0.230757,0.420929,0.263757,0.310106,0.578002,0.134422,0.269336,0.312072,0.328803,0.683237,0.093963,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
199997,0.486946,0.882191,0.466149,0.317384,0.413259,0.446430,0.736203,0.140890,0.488631,0.518777,0.417005,0.591997,0.772627,0.416671,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
199998,0.702699,0.521982,0.231086,0.837263,0.112626,0.814572,0.611743,0.387170,0.494127,0.826179,0.587442,0.718513,0.969325,0.884661,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:

"""
Optuna example that optimizes a regressor for table dataset using LightGBM tuner.
In this example, we optimize the validation rmse loss of continuous values .

"""

In [ ]:
# 交差検証のためのtrainデータとtestデータの作成

train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.33, random_state=0)


# lgbのデータセットに変更

trains = lgb.Dataset(train_x,train_y)
tests  = lgb.Dataset(test_x, test_y)

In [ ]:
# パラメータの指定 
# ソースコードコピペ https://www.inoue-kobo.com/ai_ml/lightbgm-with-optuna/index.html

# mean_squared_error 回帰問題 rmse 平均自乗誤差平方根
# num_boost_round 学習回数
# early_stopping_rounds 評価数値が5回連続で同じだった際には強制終了

params = {
    'objective': 'mean_squared_error',
    'metric': 'rmse'
}

# 学習時間計測 

tic()

model = lgb.train(params, trains, valid_sets=tests,
                    verbose_eval=False,
                    num_boost_round=1000,
                    early_stopping_rounds=50)

toc()

[I 2021-02-22 14:14:14,216] A new study created in memory with name: no-name-5701e17e-597c-41d2-84a6-c605df1fe647
min_data_in_leaf, val_score: 0.843579: 100%|##########| 5/5 [00:55<00:00, 11.13s/it]

elapsed time: 594.85111 sec

In [ ]:
predicts = model.predict(X_test)

In [ ]:
best_params = model.params

print("Best params", best_params)

for key, value in best_params.items():
  print("   {}: {}".format(key,value))

Best params {'objective': 'mean_squared_error', 'metric': 'rmse', 'feature_pre_filter': False, 'lambda_l1': 8.328257231263602e-06, 'lambda_l2': 0.00023041532101199022, 'num_leaves': 6, 'feature_fraction': 0.44800000000000006, 'bagging_fraction': 0.8447397305441933, 'bagging_freq': 1, 'min_child_samples': 20}
   objective: mean_squared_error
   metric: rmse
   feature_pre_filter: False
   lambda_l1: 8.328257231263602e-06
   lambda_l2: 0.00023041532101199022
   num_leaves: 6
   feature_fraction: 0.44800000000000006
   bagging_fraction: 0.8447397305441933
   bagging_freq: 1
   min_child_samples: 20


In [ ]:
predicts

array([7.61021479, 7.76346495, 7.67417609, ..., 7.51074684, 7.41504699,
       7.3022013 ])

In [ ]:
# 提出

sample_submission['target'] = predicts

from google.colab import files
sample_submission.to_csv("output.csv")
files.download('output.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>